In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import os
import seaborn as sn
import statsmodels.formula.api as smf
import warnings

warnings.simplefilter('ignore')
plt.style.use('ggplot')

# Tabulate station co-ordinates

List the actual station co-ordinates used for estimating river inputs for Martini (rather than relying on the approximate mappings to Andre's model inflow points).

See e-mail from Phil received 28.04.2020 at 14.34 for details.

In [2]:
# Read all tidied data files
csv_path = r'../data/tidy/martini_norway_chem_tidy.csv'
nor_wc_df = pd.read_csv(csv_path, encoding='utf-8')

csv_path = r'../data/tidy/martini_sweden_chem_tidy.csv'
swe_wc_df = pd.read_csv(csv_path, encoding='utf-8')

csv_path = r'../data/tidy/martini_denmark_chem_tidy.csv'
den_wc_df = pd.read_csv(csv_path, encoding='utf-8')

csv_path = r'../data/tidy/martini_all_daily_flows_tidy.csv'
q_df = pd.read_csv(csv_path, encoding='utf-8')

csv_path = r'../data/tidy/martini_all_river_data.csv'
all_df = pd.read_csv(csv_path, encoding='utf-8')

In [3]:
# Reformat station IDs
nor_wc_df['station_id'] = 'vm_' + nor_wc_df['station_id'].apply(lambda x: '%03d' % x)
swe_wc_df['station_id'] = 'mvm_' + swe_wc_df['station_id'].apply(lambda x: '%05d' % x)
den_wc_df['station_id'] = 'oda_' + den_wc_df['station_id'].apply(lambda x: '%s' % x)
wc_df = pd.concat([nor_wc_df, swe_wc_df, den_wc_df], axis=0, sort=False)

In [4]:
# Check site IDs are the same for wc and q
assert sorted(wc_df['station_id'].unique()) == sorted(q_df['station_id'].unique())

# Check wc_df has all required IDs for martini_all_river_data.csv
assert sorted(wc_df['station_id'].unique()) == sorted(all_df['station_id'].unique())

In [5]:
# Get co-ords
stn_df = wc_df[['station_id', 'station_name', 'lon', 'lat']].drop_duplicates().reset_index(drop=True)
stn_df.to_csv(r'../data/tidy/martini_river_stations.csv', index=False)
stn_df.head()

,station_id,station_name,lon,lat
0,vm_001,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828
1,vm_002,Glomma ved Sarpsfoss,11.134198,59.277904
2,vm_003,Mosseelva oppstrøms Mossefossen (VANU),10.670645,59.439361
3,vm_004,"Såna (Hølenelva), HOLN",10.741666,59.539246
4,vm_005,Årungen utløp,10.738662,59.701390


In [6]:
nivapy.spatial.quickmap(stn_df,
                        popup='station_id',
                        lat_col='lat',
                        lon_col='lon',
                       )